In [52]:
# !pip install pysimplegui
#pip install --force-reinstall -v "pyabf==2.3.8" #delite and reinstall the certain version
#pip install -Iv MySQL_python==1.2.2 # install the certain version
#!pip install matplotx==0.0.0

#import abf_explorer
#!pip install --upgrade numpy
import pyabf

import numpy as np

import matplotlib.pyplot as plt
#import matplotx

import statistics

import PySimpleGUI as sg

import pandas as pd

import os
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d

In [10]:
my_file = pyabf.ABF("2023_08_03_0054.abf")

In [ ]:
# GUI to choose file
tres = 0
my_file = False
while True:
    event, values = sg.Window('Get filename example', [[sg.Text('Filename')], [sg.Input(), sg.FileBrowse()], [sg.OK(), sg.Cancel()] ]).read(close=True)
    if event == 'Cancel':
        break
    try:
        my_file = pyabf.ABF(values[0])
        break
    except:
        print('Incorrect file format')

In [ ]:
# Response of both cells plots

plt.figure(figsize=(40,20))

plt.subplot (1,2,1)

for my_sweepNumber in my_file.sweepList:
    my_file.setSweep(sweepNumber = my_sweepNumber, channel = 0)
    i1, i2 = 0, int(my_file.dataRate * 1)  # plot part of the sweep
    dataX = my_file.sweepX[i1:i2] + .025 * my_sweepNumber
    dataY = my_file.sweepY[i1:i2] + 15 * my_sweepNumber
    plt.plot(dataX, dataY, color='C0', alpha=.5, label=my_sweepNumber)
# matplotx.line_labels()
plt.gca().axis('off')  # hide axes to enhance floating effect
plt.title("Response of Channel 0")

plt.subplot (1,2,2)

for my_sweepNumber in my_file.sweepList:
    my_file.setSweep(sweepNumber = my_sweepNumber, channel = 2)
    i1, i2 = int(my_file.dataRate * 1.5), int(my_file.dataRate * 2.5)  # plot part of the sweep #15000 in original for 10kHz sampling
    dataX = my_file.sweepX[i1:i2] + .025 * my_sweepNumber
    dataY = my_file.sweepY[i1:i2] + 15 * my_sweepNumber
    plt.plot(dataX, dataY, color='C0', alpha=.5, label=my_sweepNumber)
# matplotx.line_labels()
plt.gca().axis('off')  # hide axes to enhance floating effect
plt.title("Response of Channel 2")


plt.show()

In [ ]:
# Choose channel for analysis 

my_search_ch = int(input("Please enter your #channel (0 or 2!) "))

In [145]:
num_points = 40 

#define the area of interest for the cell in 1st and 2nd devices 
def TimePeriodBorders (my_file_abf):
    sample_rate = int(my_file_abf.dataRate)
    if sample_rate == 20000: 
        # Time for PreStim and Stim for Channel 0 
        my_start10 = 0 
        my_end10 = 2800    
        my_start11 = 4000 
        my_end11 = 12800 
        # Time for PreStim and Stim for Channel 2
        my_start20 = 28000 
        my_end20 = 30800 
        my_start21 = 32000 
        my_end21 = 40800 
    elif sample_rate == 10000:
        # Time for PreStim and Stim for Channel 0
        my_start10 = 0 
        my_end10 = 1400     
        my_start11 = 2000 
        my_end11 = 6400 
        # Time for PreStim and Stim for Channel 2
        my_start20 = 14000
        my_end20 = 15400
        my_start21 = 16000
        my_end21 = 20000 
    else:
        print("Check the Sampling rate!")
        return ([],[],[],[],[],[],[],[])
        
    return (my_start10, my_end10, 
            my_start11, my_end11, 
            my_start20, my_start21, 
            my_start21, my_end21)



#find the last sweep with no APs    
def sweep_AP (my_file_abf, search_ch, start, stop):
    #choose the sweep with at least 1 spikes
    for sweep_num in range(len(my_file_abf.sweepList)):
        my_file_abf.setSweep(sweep_num, channel = search_ch)
        peaks = find_peaks(my_file_abf.sweepY[start:stop], height=0)
        #print(peaks, 'My peaks')
        if len(peaks[0])>=1:
            sweep = sweep_num-1 #sweep is detected
            break
        elif len(peaks[0])==0 and sweep_num==len(my_file_abf.sweepList)-1:
            #print('Sweep is not detected!')
            sweep = len(my_file_abf.sweepList)
    #print(sweep)
    return sweep



#relative values of I and V and Rin
def IV_relative (my_file_abf, my_sweep_list,
                 voltage_channel, current_channel,
                 pre_start, pre_end, 
                 post_start, post_end, 
                 currents_delta_list = [], voltages_delta_list = [], voltage_range_ind_list = [],
                 Rin_relative_list = [], Rin_relative_norm_list = [], currents_range_list = [], voltages_range_list = []):
    
        voltages_dur_stim_list = []
    
        for my_sweepNumber in range(my_sweep_list):
            
            my_file_abf.setSweep(sweepNumber = my_sweepNumber, channel = voltage_channel)
            if voltage_channel==0:
                voltage_pre_stim = np.mean(my_file_abf.sweepY[pre_start: pre_end])
                voltage_dur_stim = np.mean(my_file_abf.sweepY[post_start: post_end])
            elif voltage_channel==2:
                voltage_pre_stim = np.mean(my_file_abf.sweepY[pre_start: pre_end])
                voltage_dur_stim = np.mean(my_file_abf.sweepY[post_start: post_end])
                
            voltage_delta = voltage_dur_stim - voltage_pre_stim
            
            my_file_abf.setSweep(sweepNumber = my_sweepNumber, channel = current_channel)
            if current_channel==1:
                current_pre_stim = np.mean(my_file_abf.sweepY[pre_start: pre_end])
                current_dur_stim = np.mean(my_file_abf.sweepY[post_start: post_end])
            elif current_channel==3:
                current_pre_stim = np.mean(my_file_abf.sweepY[pre_start: pre_end])
                current_dur_stim = np.mean(my_file_abf.sweepY[post_start: post_end])
                
            current_delta = current_dur_stim - current_pre_stim
            
            
            if abs(current_delta)<5:
                continue
                
            voltages_delta_list.append(voltage_delta)
            voltages_dur_stim_list.append(voltage_dur_stim)
            currents_delta_list.append(current_delta) 

            if voltage_dur_stim >= -80 and voltage_dur_stim <= -50 and abs(current_delta)>5:
                currents_range_list.append(current_delta)  
                voltages_range_list.append(voltage_delta)
            
        Rin_relative_list = [x / y for x,y in zip(voltages_delta_list, currents_delta_list)]
        #print(Rin_relative_list)
        Rin_relative_norm_list = np.round(Rin_relative_list/max(Rin_relative_list), 2)
        Rin_from_range = [x / y for x,y in zip(voltages_range_list, currents_range_list)]
        Rin_property = np.mean(Rin_from_range)
            
        return voltages_dur_stim_list, currents_delta_list,  Rin_relative_list, Rin_relative_norm_list, Rin_from_range, Rin_property

In [58]:
time_points = TimePeriodBorders(my_file)
last_no_AP_sweep = sweep_AP(my_file, 2, time_points[4], time_points[7])
my_relative_values = IV_relative(my_file, last_no_AP_sweep, 0, 1, time_points[0], time_points[1], time_points[2], time_points[3], [], [], [], [], [])
print(my_relative_values)

([-80.856, -50.813408, -20.707605, 9.9725485, 39.878113, 70.633286, 100.96227], [-18.760635, -12.210091, -5.275528, 3.4816437, 13.551872, 22.78759, 30.623356], [0.23202527, 0.2402927, 0.25476283, 0.34912276, 0.33983234, 0.3226183, 0.30331483], array([0.66, 0.69, 0.73, ..., 0.97, 0.92, 0.87], dtype=float32), [0.25476283, 0.34912276, 0.33983234], 0.31457266)


In [147]:
# GUI to choose folder
tres = 0
folder_path = False
while True:
    event, values = sg.Window('Choose your folder', [[sg.Text('Folder')], [sg.Input(), sg.FolderBrowse()], [sg.OK(), sg.Cancel()] ]).read(close=True)
    if event == 'Cancel':
        break
    try:
        folder_path = values[0]
        break
    except:
        print('Incorrect input')

In [151]:
my_voltage_channel, my_current_channel = 2, 3
file_names_list = []
my_Rin_range = []
my_Rin_property = []

#folder_path = 'C:/Users/user/Documents/IMBIT/05.10.2023'
file_list = os.listdir(folder_path)
#print(file_list) # List of all file names

col1 = 'voltage'
col2 = 'currents_delta_list'
col3 = 'Rin_relative_list'
col4 = 'Rin_relative_norm_list'
col5 = 'Rin_from_range'
col6 = 'Rin_property'

for filename in file_list:
    print(filename)
    file_names_list.append(filename)
    file_path = os.path.join(folder_path, filename)
    my_file = pyabf.ABF(file_path)
    
    time_points = TimePeriodBorders(my_file)
    last_no_AP_sweep = sweep_AP(my_file, 2, time_points[4], time_points[7])
    my_relative_values = IV_relative(my_file, last_no_AP_sweep, my_voltage_channel, my_current_channel, time_points[4], time_points[5], time_points[6], time_points[7], [], [], [], [], [], [], [])
    #print(my_relative_values)


    my_length = len(my_relative_values[0])
    
    my_Rin_range = my_relative_values[4]
    my_Rin_property = [my_relative_values[5]]

    while len(my_Rin_range)<my_length: 
        my_Rin_range.append(None) 

    while len(my_Rin_property)<my_length: 
        my_Rin_property.append(None)


    my_data = pd.DataFrame({col1:my_relative_values[0],
                            col2:my_relative_values[1], 
                            col3:my_relative_values[2],
                            col4:my_relative_values[3], 
                            col5:my_Rin_range,
                            col6:my_Rin_property})


    my_data.to_excel(r'C:\Users\user\Documents\IMBIT\Master\Rin relative\{}.xlsx'.format(filename) , sheet_name='All data', index=False)

print('Done!')

2023_07_13_0000.abf
2023_07_13_0004.abf
2023_07_17_0013.abf
2023_07_20_0004.abf
2023_07_24_0003.abf
2023_07_24_0014.abf
2023_07_27_0001.abf
2023_07_27_0015.abf
2023_07_27_0016.abf
2023_07_27_0020.abf
2023_08_03_0018.abf
2023_08_08_0000.abf
2023_08_08_0001.abf
2023_08_08_0009.abf
2023_08_08_0031.abf
2023_08_08_0062.abf
2023_08_08_0055.abf
2023_08_08_0056.abf
2023_08_08_0057.abf
2023_08_08_0058.abf
2023_08_08_0059.abf
2023_08_08_0063.abf
2023_08_10_0000.abf
2023_08_10_0003.abf
2023_08_10_0004.abf
2023_08_10_0042.abf
2023_08_10_0043.abf
2023_08_10_0044.abf
2023_08_10_0046.abf
2023_08_15_0000.abf
2023_08_15_0002.abf
2023_08_15_0038.abf
2023_08_15_0039.abf
2023_08_17_0002.abf
2023_08_17_0003.abf
2023_08_17_0052.abf
2023_08_17_0039.abf
2023_08_17_0053.abf
2023_08_21_0006.abf
2023_08_21_0000.abf
2023_08_21_0003.abf
2023_08_21_0004.abf
2023_08_21_0005.abf
2023_08_21_0014.abf
2023_08_24_0012.abf
2023_08_24_0000.abf
2023_08_24_0005.abf
2023_08_24_0006.abf
2023_08_24_0007.abf
2023_08_24_0009.abf
